In [91]:
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.special
import tensorflow as tf
import matplotlib as plt

In [92]:
from tensorflow.examples.tutorials.mnist import input_data

In [93]:
class neuralNet:
    
    def __init__(self,nClasses,image_size,learningRate):
        
        self.n_classes= nClasses
        self.image_size = image_size
        self.learningRate = learningRate
        self.X = tf.placeholder(dtype='float', shape=[None, self.image_size], name='X') # height, width
        self.Y = tf.placeholder(dtype='int32', name='Y')
    
    def model(self,num_nodehl1,num_nodehl2,num_nodehl3):
        
        self.n_nodes_hl1 = num_nodehl1
        self.n_nodes_hl2 = num_nodehl2
        self.n_nodes_hl3 = num_nodehl3
        
        
        self.hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([self.image_size, self.n_nodes_hl1])), 
                         'biases' : tf.Variable(tf.random_normal([self.n_nodes_hl1]))}

        self.hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([self.n_nodes_hl1, self.n_nodes_hl2])), 
                         'biases' : tf.Variable(tf.random_normal([self.n_nodes_hl2]))}

        self.hidden_layer_3 = {'weights': tf.Variable(tf.random_normal([self.n_nodes_hl2, self.n_nodes_hl3])), 
                         'biases' : tf.Variable(tf.random_normal([self.n_nodes_hl3]))}

        self.output_layer = {'weights': tf.Variable(tf.random_normal([self.n_nodes_hl3, self.n_classes])), 
                         'biases' : tf.Variable(tf.random_normal([self.n_classes]))}
        
         # X * weights + biases
        self.l1 = tf.add(tf.matmul(self.X, self.hidden_layer_1['weights']), self.hidden_layer_1['biases'])
        self.l1 = tf.nn.relu(self.l1)

        self.l2 = tf.add(tf.matmul(self.l1, self.hidden_layer_2['weights']), self.hidden_layer_2['biases'])
        self.l2 = tf.nn.relu(self.l2)

        self.l3 = tf.add(tf.matmul(self.l2, self.hidden_layer_3['weights']), self.hidden_layer_3['biases'])
        self.l3 = tf.nn.relu(self.l3)

        self.predicted_class = tf.add(tf.matmul(self.l3, self.output_layer['weights']), self.output_layer['biases'])

        return self.predicted_class


In [94]:
mnist = input_data.read_data_sets(train_dir="data/mnist", one_hot = False)
x_batch, y_true_batch = mnist.train.next_batch(24)

Extracting data/mnist\train-images-idx3-ubyte.gz
Extracting data/mnist\train-labels-idx1-ubyte.gz
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


In [95]:
image_size = 28 * 28 # input images are 28 x 28
n_classes = 10 # numbers 0 - 9 in the dataset
learningRate = 1e-3
num_nodehl1 = 700
num_nodehl2 =700
num_nodehl3 =700
train_batch_size = 128 # manipulate weights by 128 features at a time
epochs = 50
def train():
    
    neuralNet_ = neuralNet(n_classes,image_size,learningRate)
    print ("Neural Net initialized")
    predictedClasses = neuralNet_.model(num_nodehl1,num_nodehl2,num_nodehl3)
    """
    cross-entropy is a continuous function that is always positive. 
    If the predicted y equals the true y, cross-entropy equals zero.
    Cross-entropy is used in classification.
    This measures how well the model works on each image individually. 
    The softmax_cross_entropy_with_logits function makes the sum of the inputs equal to 1, normalizing the values

    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=predictedClasses, labels=neuralNet_.Y)
    cost = tf.reduce_mean( cross_entropy)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learningRate).minimize(cost)
    
    
    with tf.Session() as sess:
    # initialize the weights and biases before optimization starts
        sess.run(tf.global_variables_initializer())
        print("Session started, training begins")

        # train the network
        for iter in range(1, epochs+1):
            iter_loss = 0
            for _ in range(int(mnist.train.num_examples / train_batch_size)):
                # get the next batch of training examples. 
                # x_batch holds a batch of images
                # y_true_batch holds the true labels for x_batch
                x_batch, y_true_batch = mnist.train.next_batch(train_batch_size)

                # fd_train holds the batch in a dictionary with the named placeholders
                # in the tensorflow graph
                fd_train = {neuralNet_.X: x_batch, neuralNet_.Y: y_true_batch}
                _, i_loss = sess.run([optimizer, cost], feed_dict=fd_train)

                iter_loss += i_loss   
            print('iter', iter, 'of', epochs, 'loss: ', iter_loss)
            
        correct = tf.nn.in_top_k(predictedClasses, neuralNet_.Y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print('\ntraining set accuracy: ', accuracy.eval({neuralNet_.X: mnist.train.images, neuralNet_.Y: mnist.train.labels})) 
        print('test set accuracy: ', accuracy.eval({neuralNet_.X: mnist.test.images, neuralNet_.Y: mnist.test.labels}))

In [96]:
train()

Neural Net initialized
Session started, training begins
iter 1 of 50 loss:  1586178.25668
iter 2 of 50 loss:  81139.4549046
iter 3 of 50 loss:  39545.0357552
iter 4 of 50 loss:  22513.6690333
iter 5 of 50 loss:  13874.724625
iter 6 of 50 loss:  8052.13278171
iter 7 of 50 loss:  5049.7586467
iter 8 of 50 loss:  3174.91844182
iter 9 of 50 loss:  1847.91737552
iter 10 of 50 loss:  1154.26184219
iter 11 of 50 loss:  820.901091038
iter 12 of 50 loss:  625.041497518
iter 13 of 50 loss:  326.944234612
iter 14 of 50 loss:  203.655233412
iter 15 of 50 loss:  210.711860754
iter 16 of 50 loss:  168.008256282
iter 17 of 50 loss:  94.1999906018
iter 18 of 50 loss:  61.1429731754
iter 19 of 50 loss:  42.6163966988
iter 20 of 50 loss:  29.5169159246
iter 21 of 50 loss:  23.5825819931
iter 22 of 50 loss:  15.5916622055
iter 23 of 50 loss:  37.7761814925
iter 24 of 50 loss:  21.5687114541
iter 25 of 50 loss:  12.2069147498
iter 26 of 50 loss:  12.170218416
iter 27 of 50 loss:  8.84293152297
iter 28 of 